In [6]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import os
import random
sns.set_style("whitegrid")
import matrixprofile as mp
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
import matplotlib.patches as mpatches
import matplotlib.cm as cm
import statistics
from collections import Counter

In [7]:
# Location of DFs
tempTopDir = os.path.abspath('/Volumes/Britt_SSD/ReDoEvals3')

In [8]:
#############################
# Evaluations Section
#############################
evalsFile = os.path.join(tempTopDir, 'comboEval.parquet')
# Make evals to df
allEvalsDF = pd.read_parquet(evalsFile)

In [9]:
evalAgents = allEvalsDF.agent.unique()
print(evalAgents)

['pop700' 'pop23' 'fear2' 'fear18' 'nml3' 'nml21' 'std300' 'std2' 'std23'
 'std232']


In [5]:
print("X min")
print(allEvalsDF.pacman_coord_x.min())
print("X max")
print(allEvalsDF.pacman_coord_x.max())
print("Y min")
print(allEvalsDF.pacman_coord_y.min())
print("Y max")
print(allEvalsDF.pacman_coord_y.max())

# so 80 is halfway for x-axis, 85 for y-axis

X min
0.0
X max
158.0
Y min
0.0
Y max
168.0


In [8]:
allEvalsDF['region'] = np.where(((allEvalsDF.pacman_coord_x < 80) & (allEvalsDF.pacman_coord_y < 85)), 1, 
                                np.where(((allEvalsDF.pacman_coord_x >= 80) & (allEvalsDF.pacman_coord_y < 85)), 2,
                                        np.where(((allEvalsDF.pacman_coord_x >= 80) & (allEvalsDF.pacman_coord_y >= 85)), 3,4)))

In [83]:
# Answers for where spend most time is region with most steps
for a_index, a_num in enumerate(allEvalsDF.agentNum.unique()):
    temp = allEvalsDF[allEvalsDF['agentNum']==a_num]
    if ((evalAgents[a_index] == 'pop700') or (evalAgents[a_index] == 'pop23') or
       (evalAgents[a_index] == 'std300') or (evalAgents[a_index] == 'std232')):
        print("\n ~~~~~~~~~~~~~~~~~~")
        print(evalAgents[a_index])
        print(temp['region'].value_counts())


 ~~~~~~~~~~~~~~~~~~
pop700
3    3900
4    2471
1    1829
2    1796
Name: region, dtype: int64

 ~~~~~~~~~~~~~~~~~~
pop23
3    3035
4    2635
1    2295
2    2031
Name: region, dtype: int64

 ~~~~~~~~~~~~~~~~~~
std300
3    3693
4    3654
1    1411
2    1238
Name: region, dtype: int64

 ~~~~~~~~~~~~~~~~~~
std232
4    3269
1    2324
3    2288
2    2115
Name: region, dtype: int64


In [84]:
# Answers for where more practice needed is region with smallest score
for a_index, a_num in enumerate(allEvalsDF.agentNum.unique()):
    temp = allEvalsDF[allEvalsDF['agentNum']==a_num]
    if ((evalAgents[a_index] == 'pop700') or (evalAgents[a_index] == 'pop23') or
       (evalAgents[a_index] == 'std300') or (evalAgents[a_index] == 'std232')):
        print("\n ~~~~~~~~~~~~~~~~~~")
        dff = temp.groupby(["region"]).reward.sum().reset_index()
        print(evalAgents[a_index])
        print(dff.sort_values(by=['reward']))


 ~~~~~~~~~~~~~~~~~~
pop700
   region  reward
3       4  2180.0
1       2  2690.0
0       1  4000.0
2       3  7500.0

 ~~~~~~~~~~~~~~~~~~
pop23
   region  reward
3       4  4470.0
2       3  6710.0
1       2  8940.0
0       1  9960.0

 ~~~~~~~~~~~~~~~~~~
std300
   region  reward
1       2  3290.0
2       3  4220.0
0       1  4780.0
3       4  6320.0

 ~~~~~~~~~~~~~~~~~~
std232
   region  reward
2       3  4650.0
3       4  7430.0
0       1  8250.0
1       2  9720.0


In [22]:
allEvalsDF.columns

Index(['action_name', 'action', 'reward', 'episode_reward', 'epoch_reward',
       'total_reward', 'lives', 'end_of_episode', 'end_of_epoch', 'episode',
       'episode_step', 'epoch', 'epoch_step', 'state', 'mean_reward',
       'to_pill_one', 'pill_one_eaten', 'to_pill_two', 'pill_two_eaten',
       'to_pill_three', 'pill_three_eaten', 'to_pill_four', 'pill_four_eaten',
       'to_red_ghost', 'to_pink_ghost', 'to_blue_ghost', 'to_orange_ghost',
       'pacman_coord_x', 'pacman_coord_y', 'red_ghost_coord_x',
       'red_ghost_coord_y', 'pink_ghost_coord_x', 'pink_ghost_coord_y',
       'blue_ghost_coord_x', 'blue_ghost_coord_y', 'orange_ghost_coord_x',
       'orange_ghost_coord_y', 'pacman_direction', 'red_ghost_direction',
       'pink_ghost_direction', 'blue_ghost_direction',
       'orange_ghost_direction', 'dark_blue_ghost1_coord_x',
       'dark_blue_ghost1_coord_y', 'dark_blue_ghost2_coord_x',
       'dark_blue_ghost2_coord_y', 'dark_blue_ghost3_coord_x',
       'dark_blue_ghos

In [85]:
# Answers for Ghost Qs is Based on How many times it was closest at time of death
for a_index, a_num in enumerate(allEvalsDF.agentNum.unique()):
    temp = allEvalsDF[allEvalsDF['agentNum']==a_num]
    
    if ((evalAgents[a_index] == 'pop700') or (evalAgents[a_index] == 'pop23') or
       (evalAgents[a_index] == 'std300') or (evalAgents[a_index] == 'std232')):
        print("\n ~~~~~~~~~~~~~~~~~~")
        temp1 = temp[temp['end_of_episode']==True]

        temp2 = temp[temp['beforeLifeLoss']==True]

        temp1['closest_ghost'] = temp1[['to_red_ghost', 'to_pink_ghost', 'to_blue_ghost', 'to_orange_ghost',]].idxmin(axis=1)
        temp2['closest_ghost'] = temp2[['to_red_ghost', 'to_pink_ghost', 'to_blue_ghost', 'to_orange_ghost',]].idxmin(axis=1)

        print("\nLast step for " + str(evalAgents[a_index]))
        print(temp1['closest_ghost'].value_counts())
        print("Last few steps for " + str(evalAgents[a_index]))
        print(temp2['closest_ghost'].value_counts())


 ~~~~~~~~~~~~~~~~~~

Last step for pop700
to_red_ghost       20
to_orange_ghost    15
to_pink_ghost       9
to_blue_ghost       7
Name: closest_ghost, dtype: int64
Last few steps for pop700
to_red_ghost       128
to_orange_ghost     82
to_pink_ghost       54
to_blue_ghost       42
Name: closest_ghost, dtype: int64

 ~~~~~~~~~~~~~~~~~~

Last step for pop23
to_red_ghost       20
to_blue_ghost       6
to_orange_ghost     4
to_pink_ghost       3
Name: closest_ghost, dtype: int64
Last few steps for pop23
to_red_ghost       120
to_blue_ghost       36
to_orange_ghost     24
to_pink_ghost       18
Name: closest_ghost, dtype: int64

 ~~~~~~~~~~~~~~~~~~

Last step for std300
to_red_ghost       18
to_blue_ghost      10
to_pink_ghost       9
to_orange_ghost     8
Name: closest_ghost, dtype: int64
Last few steps for std300
to_red_ghost       108
to_blue_ghost       60
to_pink_ghost       54
to_orange_ghost     48
Name: closest_ghost, dtype: int64

 ~~~~~~~~~~~~~~~~~~

Last step for std232
to_red_g

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

In [18]:
# Answers for Which Agent Eats Power Pills Faster is Which Agent Eats last power pill of game at
# earliest step in game. 
# 'pill_one_eaten', 'pill_two_eaten', 'pill_three_eaten', 'pill_four_eaten'
for a_index, a_num in enumerate(allEvalsDF.agentNum.unique()):
    temp = allEvalsDF[allEvalsDF['agentNum']==a_num]
    temp['POPeaten'] = np.where(((temp.PP==True)), True, False)

#     temp['POPeaten'] = np.where(temp.pill_four_eaten==True, True, False)

    game_steps_list = []
    for g_index, g_num in enumerate(sorted(temp.epoch.unique())):
        temp1 = temp[temp['epoch']==g_num]
    
#         print(str(evalAgents[a_index]) + " game " + str(g_num))
        idx = temp1.where(temp1['POPeaten']).last_valid_index()
        if idx != None:
            df = temp1[temp1['POPeaten']==True]
#             print(df.head())
            if (len(df) > 3):
                game_steps_list.append(temp1['epoch_step'][idx])
#             print(game_steps_list)
    
    # Now do analysis on the entire list
    if ((evalAgents[a_index] == 'pop700') or (evalAgents[a_index] == 'pop23') or
       (evalAgents[a_index] == 'std300') or (evalAgents[a_index] == 'std232')):
        if ( len(game_steps_list) > 1):
            print("\n ~~~~~~~~~~~~~~~~~~ 4 Pills " + str(evalAgents[a_index]))
    #         print("Min")
    #         print (min(game_steps_list))
    #         print("Max")
    #         print (max(game_steps_list))
            print(len(game_steps_list))
            print("Mean")
            print (statistics.mean(game_steps_list[:-1]))
        #     print("Median")
        #     print (statistics.median(game_steps_list))
        #     print("Mode")
        #     print (Counter(game_steps_list).most_common())
        #     print("StdDev")
        #     print (statistics.stdev(game_steps_list))
        #     print("Variance")
        #     print (statistics.variance(game_steps_list))
        
    # And again for bad agents, which one gets to 3 faster
    game_steps_list = []
    for g_index, g_num in enumerate(sorted(temp.epoch.unique())):
        temp1 = temp[temp['epoch']==g_num]
    
#         print(str(evalAgents[a_index]) + " game " + str(g_num))
        idx = temp1.where(temp1['POPeaten']).last_valid_index()
        if idx != None:
            df = temp1[temp1['POPeaten']==True]
#             print(df.head())
            if (len(df) == 3):
                game_steps_list.append(temp1['epoch_step'][idx])
#             print(game_steps_list)
    
    # Now do analysis on the entire list
    if ((evalAgents[a_index] == 'pop700') or (evalAgents[a_index] == 'pop23') or
       (evalAgents[a_index] == 'std300') or (evalAgents[a_index] == 'std232')):
        if ( len(game_steps_list) > 1):
            print("\n ~~~~~~~~~~~~~~~~~~ 3 Pills " + str(evalAgents[a_index]))
    #         print("Min")
    #         print (min(game_steps_list))
    #         print("Max")
    #         print (max(game_steps_list))
            print(len(game_steps_list))
            print("Mean")
            print (statistics.mean(game_steps_list[:-1]))
        #     print("Median")
        #     print (statistics.median(game_steps_list))
        #     print("Mode")
        #     print (Counter(game_steps_list).most_common())
        #     print("StdDev")
        #     print (statistics.stdev(game_steps_list))
        #     print("Variance")
        #     print (statistics.variance(game_steps_list))
        
        
        
        
        
        # And again for bad agents, which one gets to 2 faster
    game_steps_list = []
    for g_index, g_num in enumerate(sorted(temp.epoch.unique())):
        temp1 = temp[temp['epoch']==g_num]
    
#         print(str(evalAgents[a_index]) + " game " + str(g_num))
        idx = temp1.where(temp1['POPeaten']).last_valid_index()
        if idx != None:
            df = temp1[temp1['POPeaten']==True]
#             print(df.head())
            if (len(df) == 2):
                game_steps_list.append(temp1['epoch_step'][idx])
#             print(game_steps_list)
    
    # Now do analysis on the entire list
    if ((evalAgents[a_index] == 'pop700') or (evalAgents[a_index] == 'pop23') or
       (evalAgents[a_index] == 'std300') or (evalAgents[a_index] == 'std232')):
        if ( len(game_steps_list) > 1):
            print("\n ~~~~~~~~~~~~~~~~~~ 2 Pills " + str(evalAgents[a_index]))
    #         print("Min")
    #         print (min(game_steps_list))
    #         print("Max")
    #         print (max(game_steps_list))
            print(len(game_steps_list))
            print("Mean")
            print (statistics.mean(game_steps_list[:-1]))
        #     print("Median")
        #     print (statistics.median(game_steps_list))
        #     print("Mode")
        #     print (Counter(game_steps_list).most_common())
        #     print("StdDev")
        #     print (statistics.stdev(game_steps_list))
        #     print("Variance")
        #     print (statistics.variance(game_steps_list))

    

/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  



 ~~~~~~~~~~~~~~~~~~ 2 Pills pop700
3
Mean
552.5


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  



 ~~~~~~~~~~~~~~~~~~ 4 Pills pop23
10
Mean
409.22222222222223


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p


 ~~~~~~~~~~~~~~~~~~ 3 Pills std300
3
Mean
438.5

 ~~~~~~~~~~~~~~~~~~ 2 Pills std300
7
Mean
297.8333333333333


/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p


 ~~~~~~~~~~~~~~~~~~ 4 Pills std232
6
Mean
432.0

 ~~~~~~~~~~~~~~~~~~ 3 Pills std232
3
Mean
350.5

 ~~~~~~~~~~~~~~~~~~ 2 Pills std232
2
Mean
316.0


In [13]:
# Answers for Which Agent Eats More Ghosts is Based on Mean and Std. Deviation of Ghosts Eaten and Simple Count
for a_index, a_num in enumerate(allEvalsDF.agentNum.unique()):
    temp = allEvalsDF[allEvalsDF['agentNum']==a_num]
    game_ghosts_list = []
    for g_index, g_num in enumerate(sorted(temp.epoch.unique())):
        temp1 = temp[temp['epoch']==g_num]
        bigR = len(temp1[temp1['reward']>60])
        game_ghosts_list.append(bigR)
    
    if ((evalAgents[a_index] == 'pop700') or (evalAgents[a_index] == 'pop23') or
       (evalAgents[a_index] == 'std300') or (evalAgents[a_index] == 'std232')):
        print("\n~~~~~~~~~~~~")
        print(evalAgents[a_index])
        print("max")
        print(max(game_ghosts_list[:-1]))
        print("mean")
        print(statistics.mean(game_ghosts_list[:-1]))


~~~~~~~~~~~~
pop700
max
2
mean
1.1176470588235294

~~~~~~~~~~~~
pop23
max
5
mean
4.454545454545454

~~~~~~~~~~~~
std300
max
6
mean
1.5333333333333334

~~~~~~~~~~~~
std232
max
7
mean
4.818181818181818


In [16]:
# Answers for Which Agent Lives Longer is Based on Mean and Std. Deviation of Epoch Length
for a_index, a_num in enumerate(allEvalsDF.agentNum.unique()):
    temp = allEvalsDF[allEvalsDF['agentNum']==a_num]
    
    game_length_list = []
    for g_index, g_num in enumerate(sorted(temp.epoch.unique())):
        temp1 = temp[temp['epoch']==g_num]
        gameLen = int(len(temp1))
        game_length_list.append(gameLen)
        
    
        
    if ((evalAgents[a_index] == 'pop700') or (evalAgents[a_index] == 'pop23') or
       (evalAgents[a_index] == 'std300') or (evalAgents[a_index] == 'std232')):
#         print(game_length_list[:-1])
#         print(game_length_list)
        
        print("\n~~~~~~~~~~~~")
        print(evalAgents[a_index])
        print("max")
        print(max(game_length_list[:-1]))
        print("mean")
        print(statistics.mean(game_length_list[:-1]))
        print("std")
        print(statistics.stdev(game_length_list[:-1]))


~~~~~~~~~~~~
pop700
max
940
mean
587.2941176470588
std
156.96128690933728

~~~~~~~~~~~~
pop23
max
1325
mean
895.7272727272727
std
161.1949694680891

~~~~~~~~~~~~
std300
max
808
mean
656
std
105.23646842095323

~~~~~~~~~~~~
std232
max
1013
mean
904.6363636363636
std
94.62058203929284


In [12]:
# Answers for Which Agent Preserves Power Pills is which agent more often has eaten less than four power pills
# when first life in a game is lost, that is, if life == 1, how many PP eaten?
for a_index, a_num in enumerate(allEvalsDF.agentNum.unique()):
    temp = allEvalsDF[allEvalsDF['agentNum']==a_num]
    
    game_pills_list = []
    for g_index, g_num in enumerate(sorted(temp.epoch.unique())):
        temp1 = temp[temp['epoch']==g_num]
        gameLen = int(len(temp1[temp1['PP']==True]))
        game_pills_list.append(gameLen)
        
    if ((evalAgents[a_index] == 'pop700') or (evalAgents[a_index] == 'pop23') or
       (evalAgents[a_index] == 'std300') or (evalAgents[a_index] == 'std232')):
        
        print("\n~~~~~~~~~~~~ ")
        print(evalAgents[a_index])
        print("At least one")
        count = len([i for i in game_pills_list[:-1] if i < 4]) 
        print(count)
        
        print("Exactly one")
        count = len([i for i in game_pills_list[:-1] if i == 3]) 
        print(count)


~~~~~~~~~~~~ 
pop700
At least one
17
Exactly one
1

~~~~~~~~~~~~ 
pop23
At least one
1
Exactly one
1

~~~~~~~~~~~~ 
std300
At least one
15
Exactly one
3

~~~~~~~~~~~~ 
std232
At least one
5
Exactly one
3
